In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import joblib
print(tf.__version__)


2.15.0


In [ ]:

# Load the trained LSTM model
model = tf.keras.models.load_model("./lstm/lstm_latency_model.keras")
print("Model loaded successfully!")

# Load the saved scaler
scaler = joblib.load("./lstm/minmax_scaler.pkl")
print("Scaler loaded successfully!")

# Function to compute differences
def compute_differences(df):
    df = df.copy()
    df["motion_score_diff"] = df["motion_score"].diff().fillna(0)
    df["flow_magnitude_diff"] = df["flow_magnitude"].diff().fillna(0)
    return df

# Function to prepare data for LSTM prediction
def prepare_data_for_model(df, scaler, features):
    df_normalized = df.copy()
    df_normalized[features] = scaler.transform(df[features])
    X = df_normalized[features].values
    X = X.reshape((X.shape[0], 1, X.shape[1]))
    return X

# Function to preprocess frames with a sliding window of 2 and predict incrementally
def preprocess_n_predict(frame_files, scaler, model, frame_window=2):
    if frame_window != 2:
        print("Error: This implementation is designed for a window size of 2 only.")
        return None
    
    if len(frame_files) < 2:
        print("Error: At least 2 frames are required for processing.")
        return None
    
    features = ["time_stamp", "motion_score", "motion_score_diff", "flow_magnitude", "flow_magnitude_diff"]
    
    # Process frames in sliding windows of 2
    for start_idx in range(0, len(frame_files) - 1, 1):  # Slide by 1 frame each time
        data = []
        prev_gray = None
        
        # Process the current pair of frames (window of 2)
        for i in range(start_idx, min(start_idx + frame_window, len(frame_files))):
            frame_file = frame_files[i]
            #print(f"Processing frame {i+1}/{len(frame_files)}: {os.path.basename(frame_file)}")
            frame = cv2.imread(frame_file)
            if frame is None:
                print(f"Warning: Could not read frame {frame_file}")
                continue
            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            if prev_gray is not None:
                # Compute motion metrics for the second frame in the pair
                diff = cv2.absdiff(prev_gray, gray)
                _, diff_thresh = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)
                motion_score = np.sum(diff_thresh) / 255

                flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                flow_magnitude = np.linalg.norm(flow, axis=2).mean()

                timestamp = int(os.path.basename(frame_file).split('_')[-1].split('.')[0])
                
                data.append({
                    "frame": i,
                    "file_name": os.path.basename(frame_file),
                    "motion_score": motion_score,
                    "flow_magnitude": flow_magnitude,
                    "time_stamp": timestamp
                })
            
            prev_gray = gray
            del frame
        
        # Create DataFrame for the current pair
        df = pd.DataFrame(data)
        if df.empty:
            print(f"Skipping window starting at frame {start_idx}: No valid data processed.")
            continue
        
        #print(f"Processed {len(data)} frames into DataFrame for window starting at frame {start_idx}")
        
        # Compute differences
        df = compute_differences(df)
        
        # Prepare data for model
        X = prepare_data_for_model(df, scaler, features)
        
        # Make predictions
        print("Making predictions for current window...")
        predictions = model.predict(X, verbose=1)
        predicted_labels = (predictions > 0.5).astype(int)
        
        # Print results for this window
        print("\nPrediction Results for Window:")
        print("Frame | File Name            | Motion Score | Flow Magnitude | Motion Score Diff | Flow Magnitude Diff | Predicted Latency | Confidence")
        print("-" * 130)
        for j in range(len(predictions)):
            frame_info = df.iloc[j]
            pred_label = "0ms" if predicted_labels[j] == 0 else "200ms"
            confidence = predictions[j][0]
            print(f"{frame_info['frame']:5d} | {frame_info['file_name'][:20]:20s} | {frame_info['motion_score']:12.2f} | "
                  f"{frame_info['flow_magnitude']:13.6f} | {frame_info['motion_score_diff']:17.2f} | "
                  f"{frame_info['flow_magnitude_diff']:19.6f} | {pred_label:15s} | {confidence:.6f}")
        print("\n" + "=" * 50 + "\n")  # Separator between windows
    
    return None



Model loaded successfully!
Scaler loaded successfully!


In [9]:
folder_path = "./raw_data_parsing/game_frames_200ms/game_frames_200ms"
frame_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) 
                          if f.endswith(('.png', '.jpg', '.jpeg'))])
preprocess_n_predict(frame_files, scaler, model, frame_window=2)

Making predictions for current window...
1/1 [==============================] - 0s 31ms/step

Prediction Results for Window:
Frame | File Name            | Motion Score | Flow Magnitude | Motion Score Diff | Flow Magnitude Diff | Predicted Latency | Confidence
----------------------------------------------------------------------------------------------------------------------------------
    1 | 172_19_113_65_174210 |       400.00 |      1.119061 |              0.00 |            0.000000 | 200ms           | 0.999998


Making predictions for current window...
1/1 [==============================] - 0s 32ms/step

Prediction Results for Window:
Frame | File Name            | Motion Score | Flow Magnitude | Motion Score Diff | Flow Magnitude Diff | Predicted Latency | Confidence
----------------------------------------------------------------------------------------------------------------------------------
    2 | 172_19_113_65_174210 |       453.00 |      1.629436 |              0.00 |  

KeyboardInterrupt: 